In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model, Model
import tensorflow as tf
from collections import Counter


In [ ]:

train_df=pd.read_csv('../input/landmark-recognition-2020/train.csv')
print(train_df.head())

In [ ]:
landmark_counts=pd.value_counts(train_df["landmark_id"])
landmark_counts=landmark_counts.reset_index()
landmark_counts.rename(columns={"index":'landmark_ids','landmark_id':'count'},inplace=True)
landmark_counts

In [ ]:
train_df["filename"] = train_df.id.str[0]+"/"+train_df.id.str[1]+"/"+train_df.id.str[2]+"/"+train_df.id+".jpg"
train_df["label"] = train_df.landmark_id.astype(str)
print(train_df.head(-1))

In [ ]:
landmarks=train_df.groupby(by='landmark_id').count().loc[:,'id']
landmarks

In [ ]:
from collections import Counter
import random

c = train_df.landmark_id .values

count = Counter(c).most_common(1000)
#print(type(count))
random.shuffle(count)
count
#print(len(count), count[-1])


In [ ]:
keep_labels = [i[0] for i in count]
#keep_labels
#train_df[train_df.landmark_id.isin(keep_labels)]
train_keep = train_df[train_df.landmark_id.isin(keep_labels)].sample(frac=1)
train_keep


In [ ]:
gen =ImageDataGenerator(rescale=1./255 ,
                        validation_split=0.20,
                        )

train_gen =gen.flow_from_dataframe(
    dataframe=train_keep,
    directory='../input/landmark-recognition-2020/train',
    target_size=(224, 224),
    x_col="filename",
    y_col="label",
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    seed = 42,
    subset="training",
    shuffle=True,
    validate_filenames=True)
    
val_gen = gen.flow_from_dataframe(
    dataframe=train_keep,
    directory='../input/landmark-recognition-2020/train',
    target_size=(224, 224),
    x_col="filename",
    y_col="label",
    color_mode="rgb",
    class_mode="categorical",
    batch_size=32,
    seed = 42,
    subset="validation",
    shuffle=True,
    validate_filenames=True)

In [ ]:

from tensorflow.keras import applications
from keras.applications import vgg16
base_model= keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3) )

base_model.trainable= True

#base_model.summary()

drop_out1= keras.layers.Dropout(0.3)
flatten_layer= keras.layers.Flatten()
drop_out2= keras.layers.Dropout(0.3)
output=keras.layers.Dense(1000, activation= 'sigmoid')
model=keras.Sequential([base_model,drop_out1,flatten_layer,drop_out2,output])
model.summary()

In [ ]:
#os.remove("'./model.h5'")

checkpoint_filepath ='./model.h5' 


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath,
                                                               monitor = 'val_loss',
                                                               mode = 'min',
                                                               save_best_only = True)
early_stopping= tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                                 mode='max',
                                                 patience=5,
                                                 restore_best_weights= True,
                                                 )

In [ ]:

model.compile(optimizer='Adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:

history= model.fit_generator(train_gen, steps_per_epoch = train_gen.samples // 32,validation_data=val_gen,validation_steps = val_gen.samples // 32 ,epochs=20,callbacks=[model_checkpoint_callback, early_stopping],shuffle=True )

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')

plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
submission_df=pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')
submission_df["filename"] = submission_df.id.str[0]+"/"+submission_df.id.str[1]+"/"+submission_df.id.str[2]+"/"+submission_df.id+".jpg"
submission_df

In [ ]:
test_gen = ImageDataGenerator().flow_from_dataframe(
    submission_df,
    directory="/kaggle/input/landmark-recognition-2020/test/",
    x_col="filename",
    y_col=None,
    weight_col=None,
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode=None,
    batch_size=1,
    shuffle=True,
    subset=None,
    interpolation="nearest",
    validate_filenames=True)

In [ ]:
y_pred_one_hot = checkpoint_filepath.predict_generator(test_gen, verbose=1, steps=len(submission_df))

y_pred = np.argmax(y_pred_one_hot, axis=-1)
y_prob = np.max(y_pred_one_hot, axis=-1)
print(y_pred.shape, y_prob.shape)

y_uniq = np.unique(train_keep.landmark_id.values)

y_pred = [y_uniq[Y] for Y in y_pred]

In [ ]:
for i in range(len(submission_df)):
    submission_df.loc[i, "landmarks"] = str(y_pred[i])+" "+str(y_prob[i])
submission_df = submission_df.drop(columns="filename")
submission_df.to_csv("submission.csv", index=False)
submission_df

In [ ]:

# for i in range(len(submission_df)):
#     submission_df.loc[i, "landmarks"] = str(y_pred[i])+" "+str(y_prob[i])
# submission_df.to_csv("submission.csv", index=False)
# submission_df